In [1]:
1+1

2

In [2]:
# cell 1: 라이브러리 import 및 설정
import mysql.connector
import pandas as pd
import re
from typing import List, Dict, Tuple, Optional
from dataclasses import dataclass
from enum import Enum
import json
import logging

# OpenAI API 키 설정 (환경변수 또는 직접 입력)
import os
from dotenv import load_dotenv

load_dotenv()

# 로깅 설정
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# MySQL 연결 설정 (실제 환경에 맞게 수정)
DB_CONFIG = {
    "host": os.environ["DB_HOST"],
    "user": os.environ["DB_USER"],
    "password": os.environ["DB_PASSWORD"],
    "database": os.environ["DB_NAME"],
    "charset": "utf8mb4",
}

print(f"DB_CONFIG: {DB_CONFIG}")

print("✅ 라이브러리 import 및 설정 완료")

DB_CONFIG: {'host': 'localhost', 'user': 'steve', 'password': 'doolman', 'database': 'youth_policy_db', 'charset': 'utf8mb4'}
✅ 라이브러리 import 및 설정 완료


In [3]:
# cell 2: Intent 분류 및 키워드 매핑 클래스 정의
class PolicyCategory(Enum):
    """정책 카테고리 정의"""

    SCHOLARSHIP = "장학금"
    EMPLOYMENT = "취업지원"
    STARTUP = "창업지원"
    HOUSING = "주거지원"
    LOAN = "대출지원"
    EDUCATION = "교육지원"
    WELFARE = "복지지원"
    UNKNOWN = "기타"


@dataclass
class QueryIntent:
    """질의 의도 분석 결과"""

    category: PolicyCategory
    keywords: List[str]
    filters: Dict[str, any]
    priority: int = 1


class IntentAnalyzer:
    """사용자 질의 의도 분석기"""

    def __init__(self):
        # 카테고리별 키워드 매핑 (실제로는 더 정교한 NLP 모델이나 임베딩 사용)
        self.category_keywords = {
            PolicyCategory.SCHOLARSHIP: [
                "장학금",
                "학비",
                "등록금",
                "대학생",
                "학자금",
                "학업지원",
                "교육비",
            ],
            PolicyCategory.EMPLOYMENT: [
                "취업",
                "일자리",
                "구직",
                "채용",
                "직업",
                "고용",
                "취업지원",
                "구직지원",
            ],
            PolicyCategory.STARTUP: [
                "창업",
                "스타트업",
                "사업",
                "기업",
                "창업지원",
                "사업지원",
            ],
            PolicyCategory.HOUSING: [
                "주거",
                "임대",
                "전세",
                "주택",
                "거주",
                "주거지원",
            ],
            PolicyCategory.LOAN: ["대출", "융자", "자금", "금융", "대출지원"],
            PolicyCategory.EDUCATION: [
                "교육",
                "연수",
                "훈련",
                "강의",
                "학습",
                "교육지원",
            ],
            PolicyCategory.WELFARE: ["복지", "지원금", "급여", "수당", "복지지원"],
        }

        # 문장 분리 패턴
        self.sentence_separators = [
            r"●\s*",  # 불릿 포인트
            r"•\s*",  # 다른 형태의 불릿
            r"\d+\.\s*",  # 숫자 리스트
            r"[,，]\s*",  # 쉼표
            r"[과와]\s+",  # 조사 '과', '와'
            r"[및과]\s+",  # '및', '과'
            r"그리고\s+",  # '그리고'
            r"\.\s+",  # 마침표
        ]

    def split_query(self, query: str) -> List[str]:
        """질의를 여러 개의 하위 질의로 분리"""
        # 모든 분리 패턴으로 문장 분리
        parts = [query]

        for pattern in self.sentence_separators:
            new_parts = []
            for part in parts:
                split_parts = re.split(pattern, part)
                new_parts.extend([p.strip() for p in split_parts if p.strip()])
            parts = new_parts

        # 중복 제거 및 정리
        unique_parts = []
        for part in parts:
            cleaned = re.sub(r"[^\w\s가-힣]", "", part).strip()
            if cleaned and len(cleaned) > 2:
                unique_parts.append(cleaned)

        logger.info(f"질의 분리 결과: {unique_parts}")
        return unique_parts

    def analyze_intent(self, query: str) -> QueryIntent:
        """단일 질의의 의도 분석"""
        query_lower = query.lower()

        # 카테고리별 매칭 점수 계산
        category_scores = {}
        matched_keywords = []

        for category, keywords in self.category_keywords.items():
            score = 0
            for keyword in keywords:
                if keyword in query_lower:
                    score += 1
                    matched_keywords.append(keyword)
            category_scores[category] = score

        # 최고 점수 카테고리 선택
        best_category = max(category_scores, key=category_scores.get)
        if category_scores[best_category] == 0:
            best_category = PolicyCategory.UNKNOWN

        # 추가 필터 추출
        filters = self._extract_filters(query)

        return QueryIntent(
            category=best_category,
            keywords=matched_keywords,
            filters=filters,
            priority=1,
        )

    def _extract_filters(self, query: str) -> Dict[str, any]:
        """질의에서 필터 조건 추출"""
        filters = {}

        # 연령대 추출
        age_pattern = r"(\d+)세?[\s~-]*(\d+)?세?"
        age_match = re.search(age_pattern, query)
        if age_match:
            min_age = int(age_match.group(1))
            max_age = int(age_match.group(2)) if age_match.group(2) else min_age
            filters["age_range"] = (min_age, max_age)

        # 대상 그룹 추출
        if "대학생" in query:
            filters["target_group"] = "university_student"
        elif "청년" in query:
            filters["target_group"] = "youth"
        elif "고등학생" in query:
            filters["target_group"] = "high_school_student"

        return filters


# 테스트
analyzer = IntentAnalyzer()
print("✅ Intent 분석기 초기화 완료")

✅ Intent 분석기 초기화 완료


In [4]:
# cell 3: MySQL 쿼리 생성기 클래스
class MySQLQueryBuilder:
    """MySQL 쿼리 생성기"""

    def __init__(self):
        # 기본 SELECT 컬럼 정의
        self.base_columns = [
            "plcy_no",  # 정책번호
            "plcy_nm",  # 정책명
            "lclsf_nm",  # 정책대분류명
            "mclsf_nm",  # 정책중분류명
            "plcy_expln_cn",  # 정책설명내용
            "plcy_sprt_cn",  # 정책지원내용
            "sprvsn_inst_cd_nm",  # 주관기관코드명
            "aply_url_addr",  # 신청URL주소
            "biz_prd_bgnng_ymd",  # 사업기간시작일자
            "biz_prd_end_ymd",  # 사업기간종료일자
            "sprt_trgt_min_age",  # 지원대상최소연령
            "sprt_trgt_max_age",  # 지원대상최대연령
            "frst_reg_dt",  # 최초등록일시
        ]

        # 카테고리별 쿼리 템플릿
        self.category_conditions = {
            PolicyCategory.SCHOLARSHIP: self._build_scholarship_conditions,
            PolicyCategory.EMPLOYMENT: self._build_employment_conditions,
            PolicyCategory.STARTUP: self._build_startup_conditions,
            PolicyCategory.HOUSING: self._build_housing_conditions,
            PolicyCategory.LOAN: self._build_loan_conditions,
            PolicyCategory.EDUCATION: self._build_education_conditions,
            PolicyCategory.WELFARE: self._build_welfare_conditions,
        }

    def build_query(self, intent: QueryIntent) -> str:
        """Intent를 기반으로 MySQL 쿼리 생성"""

        # SELECT 절 생성
        select_clause = f"SELECT {', '.join(self.base_columns)}"

        # FROM 절
        from_clause = "FROM youth_policies"

        # WHERE 절 생성
        where_conditions = ["1=1"]  # 기본 조건

        # 카테고리별 조건 추가
        if intent.category in self.category_conditions:
            category_conditions = self.category_conditions[intent.category](intent)
            where_conditions.extend(category_conditions)

        # 공통 필터 조건 추가
        common_conditions = self._build_common_conditions(intent)
        where_conditions.extend(common_conditions)

        # ORDER BY 절
        order_clause = self._build_order_clause(intent)

        # LIMIT 절
        limit_clause = "LIMIT 50"

        # 최종 쿼리 조합
        query = f"""
{select_clause}
{from_clause}
WHERE {' AND '.join(where_conditions)}
{order_clause}
{limit_clause}
        """.strip()

        return query

    def _build_scholarship_conditions(self, intent: QueryIntent) -> List[str]:
        """장학금 관련 조건 생성"""
        conditions = []

        # 텍스트 검색 조건
        text_conditions = [
            "MATCH(plcy_nm, plcy_expln_cn, plcy_sprt_cn) AGAINST('+장학금' IN BOOLEAN MODE)",
            "plcy_nm LIKE '%장학금%'",
            "plcy_kywd_nm LIKE '%장학금%'",
            "mclsf_nm LIKE '%장학%'",
            "plcy_expln_cn LIKE '%학비%'",
            "plcy_sprt_cn LIKE '%등록금%'",
        ]
        conditions.append(f"({' OR '.join(text_conditions)})")

        # 대상 그룹 필터
        if intent.filters.get("target_group") == "university_student":
            target_conditions = [
                "school_cd LIKE '%UNI%'",
                "plcy_expln_cn LIKE '%대학생%'",
                "plcy_sprt_cn LIKE '%대학생%'",
            ]
            conditions.append(f"({' OR '.join(target_conditions)})")

        return conditions

    def _build_employment_conditions(self, intent: QueryIntent) -> List[str]:
        """취업 지원 관련 조건 생성"""
        conditions = []

        # 텍스트 검색 조건
        text_conditions = [
            "MATCH(plcy_nm, plcy_expln_cn, plcy_sprt_cn) AGAINST('+취업 +지원' IN BOOLEAN MODE)",
            "(plcy_nm LIKE '%취업%' AND plcy_nm LIKE '%지원%')",
            "mclsf_nm LIKE '%취업%'",
            "plcy_expln_cn LIKE '%일자리%'",
            "plcy_sprt_cn LIKE '%구직%'",
        ]
        conditions.append(f"({' OR '.join(text_conditions)})")

        # 청년 대상 필터
        if intent.filters.get("target_group") == "youth":
            youth_conditions = [
                "sprt_trgt_age_lmt_yn = 'N'",
                "(sprt_trgt_min_age <= 34 AND sprt_trgt_max_age >= 19)",
            ]
            conditions.append(f"({' OR '.join(youth_conditions)})")

        return conditions

    def _build_startup_conditions(self, intent: QueryIntent) -> List[str]:
        """창업 지원 관련 조건 생성"""
        conditions = []

        text_conditions = [
            "MATCH(plcy_nm, plcy_expln_cn, plcy_sprt_cn) AGAINST('+창업 +지원' IN BOOLEAN MODE)",
            "plcy_nm LIKE '%창업%'",
            "plcy_expln_cn LIKE '%스타트업%'",
            "mclsf_nm LIKE '%창업%'",
        ]
        conditions.append(f"({' OR '.join(text_conditions)})")

        return conditions

    def _build_housing_conditions(self, intent: QueryIntent) -> List[str]:
        """주거 지원 관련 조건 생성"""
        conditions = []

        text_conditions = [
            "MATCH(plcy_nm, plcy_expln_cn, plcy_sprt_cn) AGAINST('+주거 +지원' IN BOOLEAN MODE)",
            "plcy_nm LIKE '%주거%'",
            "plcy_nm LIKE '%임대%'",
            "mclsf_nm LIKE '%주택%'",
        ]
        conditions.append(f"({' OR '.join(text_conditions)})")

        return conditions

    def _build_loan_conditions(self, intent: QueryIntent) -> List[str]:
        """대출 지원 관련 조건 생성"""
        conditions = []

        text_conditions = [
            "MATCH(plcy_nm, plcy_expln_cn, plcy_sprt_cn) AGAINST('+대출 +지원' IN BOOLEAN MODE)",
            "plcy_nm LIKE '%대출%'",
            "plcy_nm LIKE '%융자%'",
            "mclsf_nm LIKE '%금융%'",
        ]
        conditions.append(f"({' OR '.join(text_conditions)})")

        return conditions

    def _build_education_conditions(self, intent: QueryIntent) -> List[str]:
        """교육 지원 관련 조건 생성"""
        conditions = []

        text_conditions = [
            "MATCH(plcy_nm, plcy_expln_cn, plcy_sprt_cn) AGAINST('+교육 +지원' IN BOOLEAN MODE)",
            "plcy_nm LIKE '%교육%'",
            "plcy_nm LIKE '%연수%'",
            "mclsf_nm LIKE '%교육%'",
        ]
        conditions.append(f"({' OR '.join(text_conditions)})")

        return conditions

    def _build_welfare_conditions(self, intent: QueryIntent) -> List[str]:
        """복지 지원 관련 조건 생성"""
        conditions = []

        text_conditions = [
            "MATCH(plcy_nm, plcy_expln_cn, plcy_sprt_cn) AGAINST('+복지 +지원' IN BOOLEAN MODE)",
            "plcy_nm LIKE '%복지%'",
            "plcy_nm LIKE '%지원금%'",
            "mclsf_nm LIKE '%복지%'",
        ]
        conditions.append(f"({' OR '.join(text_conditions)})")

        return conditions

    def _build_common_conditions(self, intent: QueryIntent) -> List[str]:
        """공통 필터 조건 생성"""
        conditions = []

        # 승인된 정책만 조회
        conditions.append("plcy_aprv_stts_cd = 'APPROVED'")

        # 연령 범위 필터
        if "age_range" in intent.filters:
            min_age, max_age = intent.filters["age_range"]
            age_conditions = [
                "sprt_trgt_age_lmt_yn = 'N'",
                f"(sprt_trgt_min_age <= {max_age} AND sprt_trgt_max_age >= {min_age})",
            ]
            conditions.append(f"({' OR '.join(age_conditions)})")

        # 현재 진행 중인 사업만 (선택적)
        # conditions.append("(biz_prd_end_ymd IS NULL OR biz_prd_end_ymd >= CURDATE())")

        return conditions

    def _build_order_clause(self, intent: QueryIntent) -> str:
        """ORDER BY 절 생성"""
        order_parts = []

        # FULLTEXT 검색 점수 우선 정렬
        if intent.keywords:
            keyword_text = " ".join(intent.keywords)
            order_parts.append(
                f"MATCH(plcy_nm, plcy_expln_cn, plcy_sprt_cn) AGAINST('{keyword_text}' IN NATURAL LANGUAGE MODE) DESC"
            )

        # 최신 등록일 순
        order_parts.append("frst_reg_dt DESC")

        # 조회수 순 (선택적)
        # order_parts.append("inq_cnt DESC")

        return f"ORDER BY {', '.join(order_parts)}"


# 테스트
query_builder = MySQLQueryBuilder()
print("✅ MySQL 쿼리 생성기 초기화 완료")

✅ MySQL 쿼리 생성기 초기화 완료


In [5]:
# cell 4: RAG 쿼리 프로세서 (메인 클래스)
class RAGQueryProcessor:
    """RAG를 위한 쿼리 처리 메인 클래스"""

    def __init__(self, db_config: Dict[str, str]):
        self.db_config = db_config
        self.intent_analyzer = IntentAnalyzer()
        self.query_builder = MySQLQueryBuilder()
        self.connection = None

    def connect_db(self):
        """데이터베이스 연결"""
        try:
            self.connection = mysql.connector.connect(**self.db_config)
            logger.info("✅ 데이터베이스 연결 성공")
        except mysql.connector.Error as err:
            logger.error(f"❌ 데이터베이스 연결 실패: {err}")
            raise

    def disconnect_db(self):
        """데이터베이스 연결 해제"""
        if self.connection and self.connection.is_connected():
            self.connection.close()
            logger.info("✅ 데이터베이스 연결 해제")

    def process_single_query(self, query: str) -> Tuple[str, pd.DataFrame]:
        """단일 쿼리 처리"""
        # 1. Intent 분석
        intent = self.intent_analyzer.analyze_intent(query)
        logger.info(
            f"🔍 Intent 분석 결과: {intent.category.value}, 키워드: {intent.keywords}"
        )

        # 2. SQL 쿼리 생성
        sql_query = self.query_builder.build_query(intent)
        logger.info(f"📝 생성된 SQL:\n{sql_query}")

        # 3. 쿼리 실행
        try:
            df = pd.read_sql(sql_query, self.connection)
            logger.info(f"✅ 쿼리 실행 성공: {len(df)}개 결과")
            return sql_query, df
        except Exception as e:
            logger.error(f"❌ 쿼리 실행 실패: {e}")
            return sql_query, pd.DataFrame()

    def process_multiple_queries(
        self, user_query: str
    ) -> List[Tuple[str, str, pd.DataFrame]]:
        """다중 쿼리 처리"""
        # 1. 쿼리 분리
        sub_queries = self.intent_analyzer.split_query(user_query)
        logger.info(f"📋 분리된 쿼리 수: {len(sub_queries)}")

        results = []

        # 2. 각 서브 쿼리 처리
        for i, sub_query in enumerate(sub_queries, 1):
            logger.info(f"🔄 처리 중인 쿼리 {i}/{len(sub_queries)}: {sub_query}")

            sql_query, df = self.process_single_query(sub_query)
            results.append((sub_query, sql_query, df))

        return results

    def should_split_query(self, user_query: str) -> bool:
        """쿼리 분리 여부 판단"""
        # 분리 패턴 존재 여부 확인
        separators = ["●", "•", "과", "와", "및", "그리고", ",", "，"]

        for sep in separators:
            if sep in user_query:
                return True

        # 숫자 리스트 패턴 확인
        if re.search(r"\d+\.", user_query):
            return True

        return False

    def process_query(self, user_query: str) -> Dict[str, any]:
        """메인 쿼리 처리 함수"""
        logger.info(f"🚀 사용자 쿼리 처리 시작: {user_query}")

        # 데이터베이스 연결 확인
        if not self.connection or not self.connection.is_connected():
            self.connect_db()

        # 쿼리 분리 여부 판단
        if self.should_split_query(user_query):
            logger.info("📊 다중 쿼리 처리 모드")
            results = self.process_multiple_queries(user_query)

            return {
                "query_type": "multiple",
                "original_query": user_query,
                "results": [
                    {
                        "sub_query": sub_query,
                        "sql_query": sql_query,
                        "data": df,
                        "count": len(df),
                    }
                    for sub_query, sql_query, df in results
                ],
            }
        else:
            logger.info("📊 단일 쿼리 처리 모드")
            sql_query, df = self.process_single_query(user_query)

            return {
                "query_type": "single",
                "original_query": user_query,
                "sql_query": sql_query,
                "data": df,
                "count": len(df),
            }


# 테스트용 프로세서 초기화
processor = RAGQueryProcessor(DB_CONFIG)
print("✅ RAG 쿼리 프로세서 초기화 완료")

✅ RAG 쿼리 프로세서 초기화 완료


In [6]:
# cell 5: 테스트 케이스 실행
def test_query_processing():
    """쿼리 처리 테스트"""

    # 테스트 케이스 정의
    test_cases = [
        "대학생 장학금과 청년 취업 지원 프로그램 정보 알려줘",
        "● 대학생 장학금 정보 알려줘. ● 청년 취업 지원 프로그램 정보 알려줘.",
        "청년 창업 지원 프로그램 알려줘",
        "20대 주거 지원 정책 정보",
    ]

    print("🧪 테스트 케이스 실행 시작")
    print("=" * 80)

    for i, test_query in enumerate(test_cases, 1):
        print(f"\n📋 테스트 케이스 {i}: {test_query}")
        print("-" * 60)

        # Intent 분석만 테스트 (실제 DB 연결 없이)
        sub_queries = processor.intent_analyzer.split_query(test_query)
        print(f"🔍 분리된 쿼리: {sub_queries}")

        for j, sub_query in enumerate(sub_queries, 1):
            intent = processor.intent_analyzer.analyze_intent(sub_query)
            print(f"  📝 서브쿼리 {j}: {sub_query}")
            print(f"     - 카테고리: {intent.category.value}")
            print(f"     - 키워드: {intent.keywords}")
            print(f"     - 필터: {intent.filters}")

            # SQL 쿼리 생성
            sql_query = processor.query_builder.build_query(intent)
            print(f"     - 생성된 SQL:")
            print(f"       {sql_query[:200]}...")  # 처음 200자만 표시

        print("-" * 60)


# 테스트 실행
test_query_processing()

INFO:__main__:질의 분리 결과: ['대학생 장학금', '청년 취업 지원 프로그램 정보 알려줘']
INFO:__main__:질의 분리 결과: ['대학생 장학금 정보 알려줘', '청년 취업 지원 프로그램 정보 알려줘']
INFO:__main__:질의 분리 결과: ['청년 창업 지원 프로그램 알려줘']
INFO:__main__:질의 분리 결과: ['20대 주거 지원 정책 정보']


🧪 테스트 케이스 실행 시작

📋 테스트 케이스 1: 대학생 장학금과 청년 취업 지원 프로그램 정보 알려줘
------------------------------------------------------------
🔍 분리된 쿼리: ['대학생 장학금', '청년 취업 지원 프로그램 정보 알려줘']
  📝 서브쿼리 1: 대학생 장학금
     - 카테고리: 장학금
     - 키워드: ['장학금', '대학생']
     - 필터: {'target_group': 'university_student'}
     - 생성된 SQL:
       SELECT plcy_no, plcy_nm, lclsf_nm, mclsf_nm, plcy_expln_cn, plcy_sprt_cn, sprvsn_inst_cd_nm, aply_url_addr, biz_prd_bgnng_ymd, biz_prd_end_ymd, sprt_trgt_min_age, sprt_trgt_max_age, frst_reg_dt
FROM y...
  📝 서브쿼리 2: 청년 취업 지원 프로그램 정보 알려줘
     - 카테고리: 취업지원
     - 키워드: ['취업']
     - 필터: {'target_group': 'youth'}
     - 생성된 SQL:
       SELECT plcy_no, plcy_nm, lclsf_nm, mclsf_nm, plcy_expln_cn, plcy_sprt_cn, sprvsn_inst_cd_nm, aply_url_addr, biz_prd_bgnng_ymd, biz_prd_end_ymd, sprt_trgt_min_age, sprt_trgt_max_age, frst_reg_dt
FROM y...
------------------------------------------------------------

📋 테스트 케이스 2: ● 대학생 장학금 정보 알려줘. ● 청년 취업 지원 프로그램 정보 알려줘.
-----------------------------------------

In [7]:
# cell 6: 실제 데이터베이스 연결 및 쿼리 실행 (선택적)
def execute_real_queries():
    """실제 데이터베이스에서 쿼리 실행"""

    # ⚠️ 실제 데이터베이스 연결 정보가 필요합니다
    # DB_CONFIG를 실제 환경에 맞게 수정해야 합니다

    try:
        # 프로세서 초기화 및 연결
        processor.connect_db()

        # 테스트 쿼리 실행
        test_query = "대학생 장학금 정보 알려줘"
        result = processor.process_query(test_query)

        print(f"🎯 쿼리 결과:")
        print(f"   - 쿼리 타입: {result['query_type']}")
        print(f"   - 원본 쿼리: {result['original_query']}")

        if result["query_type"] == "single":
            print(f"   - 결과 수: {result['count']}")
            if result["count"] > 0:
                print(f"   - 첫 번째 결과:")
                print(result["data"].head())
        else:
            print(f"   - 서브쿼리 수: {len(result['results'])}")
            for i, sub_result in enumerate(result["results"], 1):
                print(f"   - 서브쿼리 {i}: {sub_result['sub_query']}")
                print(f"     결과 수: {sub_result['count']}")

    except Exception as e:
        print(f"❌ 데이터베이스 연결 또는 쿼리 실행 실패: {e}")
        print("💡 DB_CONFIG를 실제 환경에 맞게 수정하세요")

    finally:
        processor.disconnect_db()


# 실제 DB 연결이 가능한 경우에만 주석 해제하여 실행
execute_real_queries()
# print("💡 실제 데이터베이스 연결을 위해서는 DB_CONFIG를 수정하고 execute_real_queries() 함수를 실행하세요")

INFO:__main__:✅ 데이터베이스 연결 성공
INFO:__main__:🚀 사용자 쿼리 처리 시작: 대학생 장학금 정보 알려줘
INFO:__main__:📊 단일 쿼리 처리 모드
INFO:__main__:🔍 Intent 분석 결과: 장학금, 키워드: ['장학금', '대학생']
INFO:__main__:📝 생성된 SQL:
SELECT plcy_no, plcy_nm, lclsf_nm, mclsf_nm, plcy_expln_cn, plcy_sprt_cn, sprvsn_inst_cd_nm, aply_url_addr, biz_prd_bgnng_ymd, biz_prd_end_ymd, sprt_trgt_min_age, sprt_trgt_max_age, frst_reg_dt
FROM youth_policies
WHERE 1=1 AND (MATCH(plcy_nm, plcy_expln_cn, plcy_sprt_cn) AGAINST('+장학금' IN BOOLEAN MODE) OR plcy_nm LIKE '%장학금%' OR plcy_kywd_nm LIKE '%장학금%' OR mclsf_nm LIKE '%장학%' OR plcy_expln_cn LIKE '%학비%' OR plcy_sprt_cn LIKE '%등록금%') AND (school_cd LIKE '%UNI%' OR plcy_expln_cn LIKE '%대학생%' OR plcy_sprt_cn LIKE '%대학생%') AND plcy_aprv_stts_cd = 'APPROVED'
ORDER BY MATCH(plcy_nm, plcy_expln_cn, plcy_sprt_cn) AGAINST('장학금 대학생' IN NATURAL LANGUAGE MODE) DESC, frst_reg_dt DESC
LIMIT 50
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_33284\87117044.py:40: UserWarning: pandas only supports SQLAlchemy co

🎯 쿼리 결과:
   - 쿼리 타입: single
   - 원본 쿼리: 대학생 장학금 정보 알려줘
   - 결과 수: 0


In [8]:
# cell 7: 유틸리티 함수 - 결과 분석 및 시각화
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter


def analyze_query_results(results: Dict[str, any]):
    """쿼리 결과 분석"""

    if results["query_type"] == "single":
        df = results["data"]
        if not df.empty:
            print("📊 단일 쿼리 결과 분석:")
            print(f"   - 총 정책 수: {len(df)}")
            print(f"   - 대분류 분포:")
            category_counts = df["lclsf_nm"].value_counts()
            for category, count in category_counts.head().items():
                print(f"     {category}: {count}개")

            # 연령대 분포 (데이터가 있는 경우)
            if "sprt_trgt_min_age" in df.columns:
                age_data = df[df["sprt_trgt_min_age"].notna()]
                if not age_data.empty:
                    print(
                        f"   - 평균 최소 연령: {age_data['sprt_trgt_min_age'].mean():.1f}세"
                    )
                    print(
                        f"   - 평균 최대 연령: {age_data['sprt_trgt_max_age'].mean():.1f}세"
                    )

    else:
        print("📊 다중 쿼리 결과 분석:")
        total_results = 0
        for i, sub_result in enumerate(results["results"], 1):
            total_results += sub_result["count"]
            print(f"   - 서브쿼리 {i}: {sub_result['count']}개 결과")

        print(f"   - 전체 결과 수: {total_results}개")


def export_sql_queries(
    results: Dict[str, any], filename: str = "generated_queries.sql"
):
    """생성된 SQL 쿼리를 파일로 내보내기"""

    with open(filename, "w", encoding="utf-8") as f:
        f.write("-- 온통청년 RAG 시스템 생성 쿼리\n")
        f.write(f"-- 생성일시: {pd.Timestamp.now()}\n")
        f.write("-- " + "=" * 60 + "\n\n")

        if results["query_type"] == "single":
            f.write(f"-- 원본 쿼리: {results['original_query']}\n")
            f.write(f"{results['sql_query']}\n\n")
        else:
            f.write(f"-- 원본 쿼리: {results['original_query']}\n\n")
            for i, sub_result in enumerate(results["results"], 1):
                f.write(f"-- 서브쿼리 {i}: {sub_result['sub_query']}\n")
                f.write(f"{sub_result['sql_query']}\n\n")

    print(f"✅ SQL 쿼리가 {filename} 파일로 저장되었습니다")


# 사용 예시
print("✅ 유틸리티 함수 정의 완료")
print("📝 사용법:")
print("   - analyze_query_results(result): 쿼리 결과 분석")
print("   - export_sql_queries(result): SQL 쿼리 파일 내보내기")

✅ 유틸리티 함수 정의 완료
📝 사용법:
   - analyze_query_results(result): 쿼리 결과 분석
   - export_sql_queries(result): SQL 쿼리 파일 내보내기


In [9]:
# cell 8: 종합 테스트 및 결과 확인
def comprehensive_test():
    """종합 테스트 실행"""

    print("🚀 온통청년 RAG 쿼리 시스템 종합 테스트")
    print("=" * 80)

    # 테스트 케이스들
    test_cases = [
        {
            "query": "대학생 장학금과 청년 취업 지원 프로그램 정보 알려줘",
            "expected_type": "multiple",
            "expected_categories": ["장학금", "취업지원"],
        },
        {
            "query": "● 대학생 장학금 정보 알려줘. ● 청년 취업 지원 프로그램 정보 알려줘.",
            "expected_type": "multiple",
            "expected_categories": ["장학금", "취업지원"],
        },
        {
            "query": "청년 창업 지원 프로그램 알려줘",
            "expected_type": "single",
            "expected_categories": ["창업지원"],
        },
    ]

    for i, test_case in enumerate(test_cases, 1):
        print(f"\n📋 테스트 케이스 {i}")
        print(f"   쿼리: {test_case['query']}")
        print(f"   예상 타입: {test_case['expected_type']}")
        print(f"   예상 카테고리: {test_case['expected_categories']}")
        print("-" * 60)

        # 쿼리 분리 테스트
        is_multiple = processor.should_split_query(test_case["query"])
        actual_type = "multiple" if is_multiple else "single"

        print(f"   ✅ 쿼리 타입 판단: {actual_type}")

        # Intent 분석 테스트
        if is_multiple:
            sub_queries = processor.intent_analyzer.split_query(test_case["query"])
            print(f"   ✅ 분리된 쿼리 수: {len(sub_queries)}")

            for j, sub_query in enumerate(sub_queries, 1):
                intent = processor.intent_analyzer.analyze_intent(sub_query)
                print(f"      {j}. {sub_query}")
                print(f"         카테고리: {intent.category.value}")
                print(f"         키워드: {intent.keywords}")

                # SQL 생성 테스트
                sql_query = processor.query_builder.build_query(intent)
                print(f"         SQL 길이: {len(sql_query)} 문자")
        else:
            intent = processor.intent_analyzer.analyze_intent(test_case["query"])
            print(f"   ✅ 카테고리: {intent.category.value}")
            print(f"   ✅ 키워드: {intent.keywords}")

            # SQL 생성 테스트
            sql_query = processor.query_builder.build_query(intent)
            print(f"   ✅ SQL 길이: {len(sql_query)} 문자")

        print("-" * 60)

    print("\n🎉 종합 테스트 완료!")
    print("\n💡 다음 단계:")
    print("   1. 실제 데이터베이스 연결 설정")
    print("   2. 쿼리 성능 최적화")
    print("   3. 더 정교한 Intent 분석 모델 적용")
    print("   4. 결과 랭킹 및 필터링 로직 추가")


# 종합 테스트 실행
comprehensive_test()

INFO:__main__:질의 분리 결과: ['대학생 장학금', '청년 취업 지원 프로그램 정보 알려줘']
INFO:__main__:질의 분리 결과: ['대학생 장학금 정보 알려줘', '청년 취업 지원 프로그램 정보 알려줘']


🚀 온통청년 RAG 쿼리 시스템 종합 테스트

📋 테스트 케이스 1
   쿼리: 대학생 장학금과 청년 취업 지원 프로그램 정보 알려줘
   예상 타입: multiple
   예상 카테고리: ['장학금', '취업지원']
------------------------------------------------------------
   ✅ 쿼리 타입 판단: multiple
   ✅ 분리된 쿼리 수: 2
      1. 대학생 장학금
         카테고리: 장학금
         키워드: ['장학금', '대학생']
         SQL 길이: 692 문자
      2. 청년 취업 지원 프로그램 정보 알려줘
         카테고리: 취업지원
         키워드: ['취업']
         SQL 길이: 688 문자
------------------------------------------------------------

📋 테스트 케이스 2
   쿼리: ● 대학생 장학금 정보 알려줘. ● 청년 취업 지원 프로그램 정보 알려줘.
   예상 타입: multiple
   예상 카테고리: ['장학금', '취업지원']
------------------------------------------------------------
   ✅ 쿼리 타입 판단: multiple
   ✅ 분리된 쿼리 수: 2
      1. 대학생 장학금 정보 알려줘
         카테고리: 장학금
         키워드: ['장학금', '대학생']
         SQL 길이: 692 문자
      2. 청년 취업 지원 프로그램 정보 알려줘
         카테고리: 취업지원
         키워드: ['취업']
         SQL 길이: 688 문자
------------------------------------------------------------

📋 테스트 케이스 3
   쿼리: 청년 창업 지원 프로그램 알려줘
   예상 타입: single
   예상 카테고리: [